In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
import scipy
from scipy.optimize import curve_fit
import seaborn as sns
from matplotlib import cm
import csv
import scipy.io as sio
import pandas as pd
from scipy.special import erf
import time
import torch
import os

# plotting configuration
ratio = 1.5
figure_len, figure_width = 24*ratio, 18*ratio
title_font_size = 12*ratio
font_size_1, font_size_2 = 9*ratio, 9*ratio
legend_size = 18*ratio
line_width, tick_len = ratio, 4*ratio
marker_size = 5*ratio
plot_line_width = 2*ratio
hfont = {'fontname': 'Arial'}
marker_edge_width = 4
pal = sns.color_palette("deep")
sns.set(style='ticks')

In [2]:
# network setup
N = 30 # N * N units for each cell type
dx2deg2 = 6 * 6 # (degrees per lattice spacing) squared
center = int(((N+1)*N)/2)
c_idx, s_idx, f_idx = 465, 315, 15

l_stimulus = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85]
n_stimulus = len(l_stimulus)
tau_E, tau_P, tau_S, tau_V = 0.02, 0.01, 0.01, 0.01 # time constants
alpha = 2 # power exponent for the input-output function

# simulation setup
T = 5 # simulation time
dt = 0.001
b_additional_input = True

def calc_distance(i, j):
    """calculate the distance between two neurons,
        i and j are the indices of two neurons, i and j are from 0 to N*N-1 (31*30),
        x1 and y1 are the coordinate for the first neuron, x1 and y1 are from 0 to 30.
    """
    x1, y1 = divmod(i, N)
    x2, y2 = divmod(j, N)

    xd = np.minimum(np.abs(x1 - x2), N - np.abs(x1 - x2))
    yd = np.minimum(np.abs(y1 - y2), N - np.abs(y1 - y2))
    return np.sqrt(xd**2 + yd**2)

def calc_distance_matrix(N):
    x, y = np.arange(N), np.arange(N)
    x_grid, y_grid = np.meshgrid(x, y, indexing='ij')

    x1, x2 = np.reshape(x_grid, (-1, 1)), np.reshape(x_grid, (1, -1))
    y1, y2 = np.reshape(y_grid, (-1, 1)), np.reshape(y_grid, (1, -1))

    xd = np.minimum(np.abs(x1 - x2), N - np.abs(x1 - x2))
    yd = np.minimum(np.abs(y1 - y2), N - np.abs(y1 - y2))
    return np.sqrt(xd**2 + yd**2)

############# classes #############
# firing rates class for four different cell types: E, PV, SST, VIP
class FiringRates:
    def __init__(self):
        self.E = np.zeros(N * N)
        self.P = np.zeros(N * N)
        self.S = np.zeros(N * N)
        self.V = np.zeros(N * N)

# rate fields parameters for external input sources L4 and LM, from the data analysis file
class ParaRateFields:
    def __init__(self):
        # 4 parameters for classical stimuli with different stimulus size
        self.r1, self.r2 = np.zeros(n_stimulus), np.zeros(n_stimulus)
        self.sigma1, self.sigma2 = np.zeros(n_stimulus), np.zeros(n_stimulus)
        self.bs = 0 # baseline activity

class NetConnectivity:
    """ 1. connection strengths,
        2. width of the Gaussian connectivity for recurrent connections,
        3. biases.
    """
    def __init__(self):
        # peak of the Gaussian connectivity
        self.EE, self.EP, self.ES, self.EV, self.EL4, self.ELM, self.EX = 0, 0, 0, 0, 0, 0, 0
        self.PE, self.PP, self.PS, self.PV, self.PL4, self.PLM, self.PX = 0, 0, 0, 0, 0, 0, 0
        self.SE, self.SP, self.SS, self.SV, self.SL4, self.SLM, self.SX = 0, 0, 0, 0, 0, 0, 0
        self.VE, self.VP, self.VS, self.VV, self.VL4, self.VLM, self.VX = 0, 0, 0, 0, 0, 0, 0
        self.ELM1, self.ELM2 = 0, 0
        
        # width of the Gaussian connectivity, only the recurrent connections, values are consistent with those in mathematica
        self.sigma2EE2, self.sigma2EP2, self.sigma2ES2, self.sigma2EV2 = 2*7**2, 2*5**2, 2*7**2, 2*5**2
        self.sigma2PE2, self.sigma2PP2, self.sigma2PS2, self.sigma2PV2 = 2*5**2, 2*4**2, 2*5**2, 2*4**2
        self.sigma2SE2, self.sigma2SP2, self.sigma2SS2, self.sigma2SV2 = 2*7**2, 2*5**2, 2*7**2, 2*5**2
        self.sigma2VE2, self.sigma2VP2, self.sigma2VS2, self.sigma2VV2 = 2*5**2, 2*4**2, 2*7**2, 2*4**2
        
        # bias for four cell types, from the optimization file
        self.biasE, self.biasP, self.biasS, self.biasV = 0, 0, 0, 0

class ConnectivityMat:
    def __init__(self):
        self.EEm, self.EPm, self.ESm, self.EVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.PEm, self.PPm, self.PSm, self.PVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.SEm, self.SPm, self.SSm, self.SVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.VEm, self.VPm, self.VSm, self.VVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))

############# functions #############
def initialize_fr(fr):
    # set firing rates of different populations to their baseline values
    fr.E.fill(0.5996)
    fr.P.fill(1.35)
    fr.S.fill(0.6818)
    fr.V.fill(0.8437)

def load_optimized_networks_parameters(conn_file_path):
    data = torch.load(conn_file_path, map_location=torch.device('cpu'))
    optimized_network_tensor = data['gW']
    optimized_network = optimized_network_tensor.detach().cpu().numpy()
    optimized_network = optimized_network[:4, :]
    return optimized_network
    
def initialize_connectivity_bias(J, J_mat, opti_conn_mat, b_additional_input):
    bs = np.array([0.5996, 1.35, 0.6818, 0.8437, 0.6092, 0.5996, 0.5996, 0.5996])
    opti_conn_mat[0, :] = opti_conn_mat[0, :]/(2 * np.sqrt(bs[0]))
    opti_conn_mat[1, :] = opti_conn_mat[1, :]/(2 * np.sqrt(bs[1]))
    opti_conn_mat[2, :] = opti_conn_mat[2, :]/(2 * np.sqrt(bs[2]))
    opti_conn_mat[3, :] = opti_conn_mat[3, :]/(2 * np.sqrt(bs[3]))

    if b_additional_input:
        J.EE, J.EP, J.ES, J.EV, J.EL4, J.ELM, J.EX = opti_conn_mat[0, 0], -opti_conn_mat[0, 1], -opti_conn_mat[0, 2], -opti_conn_mat[0, 3], opti_conn_mat[0, 4], opti_conn_mat[0, 5], opti_conn_mat[0, 7]
        J.PE, J.PP, J.PS, J.PV, J.PL4, J.PLM, J.PX = opti_conn_mat[1, 0], -opti_conn_mat[1, 1], -opti_conn_mat[1, 2], -opti_conn_mat[1, 3], opti_conn_mat[1, 4], opti_conn_mat[1, 5], opti_conn_mat[1, 7]
        J.SE, J.SP, J.SS, J.SV, J.SL4, J.SLM, J.SX = opti_conn_mat[2, 0], -opti_conn_mat[2, 1], -opti_conn_mat[2, 2], -opti_conn_mat[2, 3], opti_conn_mat[2, 4], opti_conn_mat[2, 5], opti_conn_mat[2, 7]
        J.VE, J.VP, J.VS, J.VV, J.VL4, J.VLM, J.VX = opti_conn_mat[3, 0], -opti_conn_mat[3, 1], -opti_conn_mat[3, 2], -opti_conn_mat[3, 3], opti_conn_mat[3, 4], opti_conn_mat[3, 5], opti_conn_mat[3, 7]
        
        bs = np.array([0.5996, 1.35, 0.6818, 0.8437, 0.6092, 0.5996, 0.5996, 0.5996])
        sqrt_bs_recurrent = np.sqrt(bs[:4])
        sign_vector = np.array([+1, -1, -1, -1, +1, +1, +1, -1])
        inputs = opti_conn_mat @ (sign_vector * bs)
        T = sqrt_bs_recurrent - inputs
    else:
        J.EE, J.EP, J.ES, J.EV, J.EL4, J.ELM, J.EX = opti_conn_mat[0, 0], -opti_conn_mat[0, 1], -opti_conn_mat[0, 2], -opti_conn_mat[0, 3], opti_conn_mat[0, 4], opti_conn_mat[0, 5], 0
        J.PE, J.PP, J.PS, J.PV, J.PL4, J.PLM, J.PX = opti_conn_mat[1, 0], -opti_conn_mat[1, 1], -opti_conn_mat[1, 2], -opti_conn_mat[1, 3], opti_conn_mat[1, 4], opti_conn_mat[1, 5], 0
        J.SE, J.SP, J.SS, J.SV, J.SL4, J.SLM, J.SX = opti_conn_mat[2, 0], -opti_conn_mat[2, 1], -opti_conn_mat[2, 2], -opti_conn_mat[2, 3], opti_conn_mat[2, 4], opti_conn_mat[2, 5], 0
        J.VE, J.VP, J.VS, J.VV, J.VL4, J.VLM, J.VX = opti_conn_mat[3, 0], -opti_conn_mat[3, 1], -opti_conn_mat[3, 2], -opti_conn_mat[3, 3], opti_conn_mat[3, 4], opti_conn_mat[3, 5], 0
        bs = np.array([0.5996, 1.35, 0.6818, 0.8437, 0.6092, 0.5996])
        sqrt_bs_recurrent = np.sqrt(bs[:4])
        sign_vector = np.array([+1, -1, -1, -1, +1, +1])
        inputs = opti_conn_mat @ (sign_vector * bs)
        T = sqrt_bs_recurrent - inputs

    J.biasE, J.biasP, J.biasS, J.biasV = T[0], T[1], T[2], T[3]

    # L4 related connectivity parameters
    sigma2L3L42 = 2.0 * 7.0**2
    J.EL4 = J.EL4 / (np.pi * sigma2L3L42)
    J.PL4 = J.PL4 / (np.pi * sigma2L3L42)
    J.SL4 = J.SL4 / (np.pi * sigma2L3L42)
    J.VL4 = J.VL4 / (np.pi * sigma2L3L42)
    
    # LM and X related connectivity parameters
    
    p_ELM, q_ELM = 1, 0 # for the gaussian version
    sigma2ELM2_1, sigma2ELM2_2 = 2.0 * 15.0**2, 2.0 * 15.0**2 # for the gaussian version

    sigma2PLM2 = 2.0 * 15.0**2
    
    J.ELM1, J.ELM2 = 0, 0
    J.ELM1 = (J.ELM * p_ELM)/ (np.pi * sigma2ELM2_1)
    J.ELM2 = (J.ELM * q_ELM)/ (np.pi * sigma2ELM2_2)
    J.PLM = J.PLM / (np.pi * sigma2PLM2)
    J.SLM = J.SLM / (np.pi * sigma2PLM2)
    J.VLM = J.VLM / (np.pi * sigma2PLM2)
    
    sigma2SX2 = 2.0 * 15.0**2
    J.EX = J.EX / (np.pi * sigma2SX2)
    J.PX = J.PX / (np.pi * sigma2SX2)
    J.SX = J.SX / (np.pi * sigma2SX2)
    J.VX = J.VX / (np.pi * sigma2SX2)
    
    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    J_mat.EEm = J.EE * np.exp(-d2_matrix / J.sigma2EE2) / (np.pi * J.sigma2EE2)
    J_mat.EPm = J.EP * np.exp(-d2_matrix / J.sigma2EP2) / (np.pi * J.sigma2EP2)
    J_mat.ESm = J.ES * np.exp(-d2_matrix / J.sigma2ES2) / (np.pi * J.sigma2ES2)
    J_mat.EVm = J.EV * np.exp(-d2_matrix / J.sigma2EV2) / (np.pi * J.sigma2EV2)

    J_mat.PEm = J.PE * np.exp(-d2_matrix / J.sigma2PE2) / (np.pi * J.sigma2PE2)
    J_mat.PPm = J.PP * np.exp(-d2_matrix / J.sigma2PP2) / (np.pi * J.sigma2PP2)
    J_mat.PSm = J.PS * np.exp(-d2_matrix / J.sigma2PS2) / (np.pi * J.sigma2PS2)
    J_mat.PVm = J.PV * np.exp(-d2_matrix / J.sigma2PV2) / (np.pi * J.sigma2PV2)

    J_mat.SEm = J.SE * np.exp(-d2_matrix / J.sigma2SE2) / (np.pi * J.sigma2SE2)
    J_mat.SPm = J.SP * np.exp(-d2_matrix / J.sigma2SP2) / (np.pi * J.sigma2SP2)
    J_mat.SSm = J.SS * np.exp(-d2_matrix / J.sigma2SS2) / (np.pi * J.sigma2SS2)
    J_mat.SVm = J.SV * np.exp(-d2_matrix / J.sigma2SV2) / (np.pi * J.sigma2SV2)

    J_mat.VEm = J.VE * np.exp(-d2_matrix / J.sigma2VE2) / (np.pi * J.sigma2VE2)
    J_mat.VPm = J.VP * np.exp(-d2_matrix / J.sigma2VP2) / (np.pi * J.sigma2VP2)
    J_mat.VSm = J.VS * np.exp(-d2_matrix / J.sigma2VS2) / (np.pi * J.sigma2VS2)
    J_mat.VVm = J.VV * np.exp(-d2_matrix / J.sigma2VV2) / (np.pi * J.sigma2VV2)

def read_rf_parameters_X(para_X, stim_type):
    if stim_type == 0:
        para_X.r1 = 1/8 * np.sqrt(np.asarray([0, 5, 15, 25, 35, 45, 55, 65, 75, 85]))
        para_X.sigma1 = np.ones(10) * 15
        para_X.sigma2 = np.ones(10) * 15
        para_X.bs = 0
    else:
        para_X.r1 = np.zeros(9)
        para_X.sigma1 = np.ones(9) * 15
        para_X.sigma2 = np.ones(9) * 15
        para_X.bs = 0

def define_L4_inputs(para_L4, H_L4, stim_type, stim_idx):
    # load L4 rate field, replace R_L4
    
    rate_field_inputs_L4 = sio.loadmat('data/rate_field_inputs_L4.mat')['rate_field_inputs']
    if stim_idx == 0:
        R_L4 = rate_field_inputs_L4[stim_type*9+stim_idx, :, :].reshape((900)) * 0 + 0.6092
    else:
        R_L4 = rate_field_inputs_L4[stim_type*9+stim_idx-1, :, :].reshape((900))
    
    sigma2L3L42 = 2.0 * 7.0**2

    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_matrix = np.exp(-d2_matrix / sigma2L3L42)
    H_L4 = dx2deg2 * H_matrix @ R_L4

    return H_L4
            
def define_LM_inputs(para_LM, H_LM, H_ELM1, H_ELM2, stim_type, stim_idx):
    # load LM rate field, replace R_LM
    rate_field_inputs_LM = sio.loadmat('data/rate_field_inputs_LM.mat')['rate_field_inputs']
    if stim_idx == 0:
        R_LM = rate_field_inputs_LM[stim_type*9+stim_idx, :, :].reshape((900)) * 0 + 0.5996
    else:
        R_LM = rate_field_inputs_LM[stim_type*9+stim_idx-1, :, :].reshape((900))
    
    sigma2ELM2_1, sigma2ELM2_2 = 2.0 * 15.0**2, 2.0 * 15.0**2  # for the gaussian version
    sigma2PLM2 = 2.0 * 15.0**2

    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_LM_matrix = np.exp(-d2_matrix / sigma2PLM2)
    H_ELM1_matrix = np.exp(-d2_matrix / sigma2ELM2_1)
    H_ELM2_matrix = np.exp(-d2_matrix / sigma2ELM2_2)

    H_LM = dx2deg2 * H_LM_matrix @ R_LM
    H_ELM1 = dx2deg2 * H_ELM1_matrix @ R_LM
    H_ELM2 = dx2deg2 * H_ELM2_matrix @ R_LM

    return H_LM, H_ELM1, H_ELM2

        
def define_X_inputs(para_X, H_X, stim_type, stim_idx):
    sigma2X2 = 2.0 * para_X.sigma1[stim_idx]**2
    
    distances_to_center_sq = np.array([calc_distance(i, center) for i in range(N * N)])**2
    R_X = para_X.r1[stim_idx] * np.exp(-(distances_to_center_sq*dx2deg2/sigma2X2))

    sigma2SX2 = 2.0 * 15.0**2
    
    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_X_matrix = np.exp(-d2_matrix / sigma2SX2)

    H_X = dx2deg2 * H_X_matrix @ R_X
        
    return H_X

def run_SSN(fr, J, J_mat, H_L4, H_ELM1, H_ELM2, H_LM, H_X, s_manipulation, k): # Checked, fine
    RecInputE = np.matmul(J_mat.EEm, fr.E) + np.matmul(J_mat.EPm, fr.P) + np.matmul(J_mat.ESm, fr.S) + np.matmul(J_mat.EVm, fr.V)
    RecInputP = np.matmul(J_mat.PEm, fr.E) + np.matmul(J_mat.PPm, fr.P) + np.matmul(J_mat.PSm, fr.S) + np.matmul(J_mat.PVm, fr.V)
    RecInputS = np.matmul(J_mat.SEm, fr.E) + np.matmul(J_mat.SPm, fr.P) + np.matmul(J_mat.SSm, fr.S) + np.matmul(J_mat.SVm, fr.V)
    RecInputV = np.matmul(J_mat.VEm, fr.E) + np.matmul(J_mat.VPm, fr.P) + np.matmul(J_mat.VSm, fr.S) + np.matmul(J_mat.VVm, fr.V)
    
    TotInputE = J.EL4 * H_L4 + J.ELM1 * H_ELM1 - J.ELM2 * H_ELM2 + J.EX * H_X + dx2deg2 * RecInputE + J.biasE
    TotInputP = J.PL4 * H_L4 + J.PLM * H_LM + J.PX * H_X + dx2deg2 * RecInputP + J.biasP
    TotInputS = J.SL4 * H_L4 + J.SLM * H_LM + J.SX * H_X + dx2deg2 * RecInputS + J.biasS
    TotInputV = J.VL4 * H_L4 + J.VLM * H_LM + J.VX * H_X + dx2deg2 * RecInputV + J.biasV  
    
    fr.E += dt * ((-fr.E + np.maximum(0.0, TotInputE) ** alpha) / tau_E)
    
    if k >= 4000:
        if s_manipulation == 'uniform_inh_perturbation':
            fr.P += dt * ((-fr.P + np.maximum(0.0, TotInputP+0.001) ** alpha) / tau_P)
            fr.S += dt * ((-fr.S + np.maximum(0.0, TotInputS+0.001) ** alpha) / tau_S)
            fr.V += dt * ((-fr.V + np.maximum(0.0, TotInputV+0.001) ** alpha) / tau_V)
        elif s_manipulation == 'uniform_PV_perturbation':
            fr.P += dt * ((-fr.P + np.maximum(0.0, TotInputP+0.001) ** alpha) / tau_P)
            fr.S += dt * ((-fr.S + np.maximum(0.0, TotInputS) ** alpha) / tau_S)
            fr.V += dt * ((-fr.V + np.maximum(0.0, TotInputV) ** alpha) / tau_V)
        elif s_manipulation == 'uniform_SST_perturbation':
            fr.P += dt * ((-fr.P + np.maximum(0.0, TotInputP) ** alpha) / tau_P)
            fr.S += dt * ((-fr.S + np.maximum(0.0, TotInputS+0.001) ** alpha) / tau_S)
            fr.V += dt * ((-fr.V + np.maximum(0.0, TotInputV) ** alpha) / tau_V)
        elif s_manipulation == 'uniform_VIP_perturbation':
            fr.P += dt * ((-fr.P + np.maximum(0.0, TotInputP) ** alpha) / tau_P)
            fr.S += dt * ((-fr.S + np.maximum(0.0, TotInputS) ** alpha) / tau_S)
            fr.V += dt * ((-fr.V + np.maximum(0.0, TotInputV+0.001) ** alpha) / tau_V)
    else:
        fr.P += dt * ((-fr.P + np.maximum(0.0, TotInputP) ** alpha) / tau_P)
        fr.S += dt * ((-fr.S + np.maximum(0.0, TotInputS) ** alpha) / tau_S)
        fr.V += dt * ((-fr.V + np.maximum(0.0, TotInputV) ** alpha) / tau_V)

In [3]:
fr = FiringRates() # initialize firing rates of the network
para_L4, para_LM, para_X = ParaRateFields(), ParaRateFields(), ParaRateFields()
J = NetConnectivity()
J_mat = ConnectivityMat() # initialize the connectivity matrices

# currents from different locations in L4, LM, and X
H_L4, H_ELM1, H_ELM2, H_LM, H_X = np.zeros(N * N), np.zeros(N * N), np.zeros(N * N), np.zeros(N * N), np.zeros(N * N)

l_config = ['final_gaussian_classical']
l_selected_indices = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

n_stim_type = 1
l_s_manipulation = ['uniform_PV_perturbation', 'uniform_SST_perturbation', 'uniform_VIP_perturbation']

startpoint = 3000
for s_manipulation in l_s_manipulation:
    
    print(s_manipulation)
    fr_E_mat = np.zeros((len(l_selected_indices[0]), len(l_stimulus) * n_stim_type, 2000)) * np.nan
    fr_P_mat = np.zeros((len(l_selected_indices[0]), len(l_stimulus) * n_stim_type, 2000)) * np.nan
    fr_S_mat = np.zeros((len(l_selected_indices[0]), len(l_stimulus) * n_stim_type, 2000)) * np.nan
    fr_V_mat = np.zeros((len(l_selected_indices[0]), len(l_stimulus) * n_stim_type, 2000)) * np.nan

    for config_idx in range(len(l_config)):
        config = l_config[config_idx]
        folder_path = 'models/optimized_files_' + config + '/'
        model_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f != '.DS_Store']
        sorted_files = sorted(model_names, key=lambda name: float(name.replace('.pt', '')))
        sorted_numbers = sorted([float(f.replace('.pt', '')) for f in sorted_files])
        selected_indices = l_selected_indices[config_idx]

        for model_name_idx in selected_indices:
            model_name = sorted_files[model_name_idx]
            print(model_name)
            conn_file_path = folder_path + model_name
            optimized_network = load_optimized_networks_parameters(conn_file_path)

            initialize_connectivity_bias(J, J_mat, optimized_network, b_additional_input)  # initialize connectivity and bias

            for stim_type in range(n_stim_type):
                if stim_type == 0:
                    s_stim_type = 'classical'
                else:
                    s_stim_type = 'inverse'

                read_rf_parameters_X(para_X, stim_type)  # read the rate field parameters for this subsampled rate field

                for stim_idx in range(len(l_stimulus)):
                    initialize_fr(fr)
                    print("stimulus: " + str(l_stimulus[stim_idx]))

                    H_L4 = define_L4_inputs(para_L4, H_L4, stim_type, stim_idx)
                    H_LM, H_ELM1, H_ELM2 = define_LM_inputs(para_LM, H_LM, H_ELM1, H_ELM2, stim_type, stim_idx)
                    H_X = define_X_inputs(para_X, H_X, stim_type, stim_idx)

                    for k in range(int(T / dt)):
                        run_SSN(fr, J, J_mat, H_L4, H_ELM1, H_ELM2, H_LM, H_X, s_manipulation, k)
                        
                        if k == startpoint:
                            fr_E_bs = np.copy(fr.E)
                            fr_P_bs = np.copy(fr.P)
                            fr_S_bs = np.copy(fr.S)
                            fr_V_bs = np.copy(fr.V)
                        
                        if k >= startpoint:
                            fr_E_mat[model_name_idx, int((len(l_stimulus))*stim_type+stim_idx), int(k-startpoint)] = np.mean(fr.E) # some index
                            fr_P_mat[model_name_idx, int((len(l_stimulus))*stim_type+stim_idx), int(k-startpoint)] = np.dot(fr.P - fr_P_bs, np.ones(900))
                            fr_S_mat[model_name_idx, int((len(l_stimulus))*stim_type+stim_idx), int(k-startpoint)] = np.dot(fr.S - fr_S_bs, np.ones(900))
                            fr_V_mat[model_name_idx, int((len(l_stimulus))*stim_type+stim_idx), int(k-startpoint)] = np.dot(fr.V - fr_V_bs, np.ones(900))

        sio.savemat('simulation_data_folder/fr_E_' + s_manipulation + '_' + config + '.mat', mdict={'E': fr_E_mat})
        sio.savemat('simulation_data_folder/fr_P_' + s_manipulation + '_' + config + '.mat', mdict={'P': fr_P_mat})
        sio.savemat('simulation_data_folder/fr_S_' + s_manipulation + '_' + config + '.mat', mdict={'S': fr_S_mat})
        sio.savemat('simulation_data_folder/fr_V_' + s_manipulation + '_' + config + '.mat', mdict={'V': fr_V_mat})

uniform_PV_perturbation
2.9814842343e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimulus: 45
stimulus: 55
stimulus: 65
stimulus: 75
stimulus: 85
3.0061629415e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimulus: 45
stimulus: 55
stimulus: 65
stimulus: 75
stimulus: 85
3.0643698573e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimulus: 45
stimulus: 55
stimulus: 65
stimulus: 75
stimulus: 85
3.0751633644e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimulus: 45
stimulus: 55
stimulus: 65
stimulus: 75
stimulus: 85
3.0905631185e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimulus: 45
stimulus: 55
stimulus: 65
stimulus: 75
stimulus: 85
3.1116414070e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimulus: 45
stimulus: 55
stimulus: 65
stimulus: 75
stimulus: 85
3.1124791503e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimu